<a href="https://colab.research.google.com/github/DGkurnia/sentiment/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Nama: Derfansyah Guswiranata Kurnia
* Identitas Dicoding : dgkurnia
* email dicoding : weerakurnia@gmail.com
* Alamat: Jalan Malaka Hijau No 39 Perumahan Malaka Country Estate

#Pembaruan

In [1]:
#perkembangan google colab
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,234 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,646 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/u

In [2]:
#request dan pandas
!pip install requests pandas

In [3]:
#instalasi sastarawi
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.6 MB/s eta 0:00:00


In [4]:
#instalasi wordcloud
!pip install wordcloud

In [5]:
#instalasi backoff
!pip install backoff aiohttp

# Impor Perpustakaan

In [6]:
#untuk akses ke google drive (bila diperlukan)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
#instalasi perpustakaan
import requests
import json
#Bagian Pandas
import pandas as pd
pd.options.mode.chained_assignment = None #pengaturan peringatan 'rantai'
import csv
#ke manajemen

In [27]:
#manajemen data dasar
import numpy as np
seed = 0 #reprodubilitas
np.random.seed(seed)
#ke plt

In [28]:
#Bagian plt
import matplotlib.pyplot as plt
import datetime as dt #jika ada tanggal
#perpustakaan untuk interpolasi
from scipy.interpolate import interp1d
#waktu
import time
#ke sastawi, NLTK, dan wordcloud

In [16]:
#Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #bagian 'stemmer factory'
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [15]:
#sastrawi dan NITK [NITK]
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.
from nltk.corpus import stopwords  # Mengimpor modul stopwords dari pustaka NLTK.
from nltk.tokenize import word_tokenize  # Mengimpor fungsi word_tokenize dari modul tokenize dalam pustaka NLTK.

#impor wordcloud
from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks
#ke sastrawi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Impor Teknik

In [24]:
#pemisahan data dengan 'train test split'
from sklearn.model_selection import train_test_split

In [10]:
#Teknik untuk inspeksi: perpustakaan untuk klasifikasi
from sklearn.ensemble import RandomForestClassifier as rfclasifi #teknik 'randomfprest'
from sklearn.tree import DecisionTreeClassifier #teknik 'keputusan'
from sklearn.svm import SVC #teknik SVC
from sklearn.linear_model import LogisticRegression

In [43]:
#Bagian re IO bentuk string
import re
from io import StringIO

In [29]:
#metrik penilaian (akurasi, lenyap/loss, nilai f1, nilai recall)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Persiapan Ekstraksi situs

##Pendaftaran situs

In [11]:
#deklarasi 'URL target' dan asal URL
turldoma = 'https://gql.tokopedia.com/graphql/productReviewList'
asal = 'https://www.tokopedia.com' #semua dari Tokopedia
#ke pendaftaran URL

In [47]:
"""
#Daftar 'URL referensi' dan 'cookie' di Tokopedia
#daftar URL referensi
"""
urllist = ["https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-jazlyn-black-jazlyn-black-s/review", "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gribson-rayon-navy-s/review",
"https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-lavrenti-white-lavrenti-white-s/review", "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-nessie-dark-green-s/review",
"https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gruney-katun-red-m/review", "https://www.tokopedia.com/erigo/kaos-erigo-t-shirt-oversize-antelope-cotton-combed-black-m/review"]
#Bagian Pendaftaran 'cookie'
cooklist = ["_UUID_NONLOGIN_=51509898fc52958ba09660d9b6d42e74; _UUID_NONLOGIN_.sig=tuUBVA3nJOrWzWPfGXPe0ChiD9w; _gid=GA1.2.1360322750.1737634767; _gcl_au=1.1.1168896657.1737634767; DID=a49d25541bbf6b98d7601fda0a58ed5d1ffe8cbde2b1a5fad5e2d5931895e6accf724486311af4ebf8d7f058dd950d4b; DID_JS=YTQ5ZDI1NTQxYmJmNmI5OGQ3NjAxZmRhMGE1OGVkNWQxZmZlOGNiZGUyYjFhNWZhZDVlMmQ1OTMxODk1ZTZhY2NmNzI0NDg2MzExYWY0ZWJmOGQ3ZjA1OGRkOTUwZDRi47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _SID_Tokopedia_=lI_oSYY2b4kIa2oODs-BBIUxMyP5OF_62gPn53Rulw_USlSlfA8vBbnFjjTrHTsk8qpxkMJ5WCWdGQsIlmIovsYBA_NGHc8ENhAr1rAD-g0xtZu-DPSR5r0FB6-227Rc; ak_bmsc=660754FBF339FACB642B73EE4910C848~000000000000000000000000000000~YAAQ3i2pyp2/douUAQAA4GMakxpzGI5iEKO6XPqLDafHgckcG1PutzprYH3zTlx7PGCpNLflgdj7hqkbpIgwRS1Sbd3KLj72Lxa8MxdtDDePzpnyZxKnC0c3mHFGoDtYlur0bD1XHr42aXthzfo9g0H4GP+RKddacSYrvdDJ1IFZoT1mi4tuEaqJQp96EUr65QnkqxnDepVdJhz+2OgdldeqCFyES6FJERdMKkMJ8Z9tPqQkicxyPKg9YQmHlQe83REDCPa66FIoOP7egYoltKIJBol1BPBGBPxlbBvMWPcRTJK/jWvWtOtoBK1liRPsk/aCh9i7pHNznlZkvapWK+1LvclLscLtJZRq7crBG9+D9qcwMaKWHEEO2YEKrL+QdJy8Rnbf3WJZSuDmnfYHKSVlm35egjekouh/2aiCWGBgGkbDVM5+8QOejjG0EUugA+y4SsHmDsenNbSszEW6; _tt_enable_cookie=1; _ttp=gyVtLmTmhCKYRGPVXG7sNm5l0nV.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=JMCm5TEOQtGCKsOjZsngIA; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImRlNTNiMzUxLTEwZmMtNTRkZi1iNzJhLTA1ODQzYTZhMGExOCIsImNyZWF0ZWQiOjE3Mzc2MzQ3NzQwMDMsImV4aXN0aW5nIjpmYWxzZX0=; _hjSession_714968=eyJpZCI6ImE3ODk2YWNiLWEwMjEtNDQwOS1iYjI3LTdmNGY5MGFhMWE3YiIsImMiOjE3Mzc2MzQ3NzQwMDMsInMiOjEsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxfQ==; bm_sv=30C3C116672B9FACAA44837DC2769CE6~YAAQ1i2pyr/ZWIuUAQAAuvgakxoKk2UfT2kgI5DG3EnV7RnbKRJLSkZZbSzBi0cjpzvAeregCYj3HCslyNabwcXG3KR+8eKnC+O3WRr2G7fH7t2ReHQr/E1zlmQ/ES7uRyfWoRzc1CrrXjINiuv5H3Ak806mKFa1Rv7uM0J4IELZAW7RscfcC4WmbgnoVkS91A9EJd8Ut3ZG5fS+jaAW3UPqGSa+CGruyK9j4pCYikQ0vnsBbkv5WzT6HHvVzNYGZ49K~1; _gat_gtag_UA_9801603_6=1; _dc_gtm_UA-9801603-6=1; FPF=1; uide=k81BMzhOXkLjERTW5+b+2EIxjcm9BQ5+97/VDzpmYF3rZUiy; _abck=13CB010E67315CE330AA82F7B2562336~0~YAAQDunOF+m9aouUAQAAiP0akw0Ce9K8hRO1zIREiLJUu4Aq4qWrfrF7U807yX3Xl6hIi82CdskV8PDJRf0kmxZUQMf3fLW1emlxMpGKalRPpu2c1EygvmUcr1JOOG2S4TcPX3lscKKMrl4V4c7BUA/xDCgRPIcQbfG0z6wlgv6nM6L99wtD8Gd6lu7hR7C990O8tB1TYcJRBGeGURHLHqnQv41MAWQ9Iy5Gqf2bMyy6kEgS7U9WiluzgBzbIwntsUVwUwqIkifO5rPSMmbgEFKprDCAbVbsaLlooQQHz/U3+uga9Y3rcNgAgwwKT4nlUzmoXdohWkQffJaVZVVp5ZntvBzPVNLJbO3Vg7AbQe52X+PsjP0ee4xXJ0t9DcxijZsmkG1xhXkJ+hIGT9gr2Ctb8v47NINQ1nr4VAU8hE6B37jhUmUAFW/euqDhY6sdh15+56so4teE/yXdRjaoouX9I0BRZVRSXZE7R3TntAjnQ4KHqkGeSBo8XNZeIo35AbNUpsDji3xIiWtQsaaX1zYCanJl81oeZUGGD5ZredFi5BaPsztag4EtSQEcClhC3CWMxmF3cE4K4HieNJKfqcnmfb/stlOq3q3zlkVvrsKQ50Ox9gW5+0dOIa616uq5Stj4FjqmfrGAALv75hUUmBylx/Vsp2ZMWnk=~-1~-1~-1; bm_sz=2DAD631839A27230FD189145CC677C8A~YAAQDunOF+q9aouUAQAAiP0akxp8gSwLwkHptWgc1vOURdOWx+0I96UxoLGtN6l5QEd3orcVZtLfV7OTdhnSU4JTwJpFnq0wc5iHLE/HeM7Jn93Oi9ObJeTmZvzVXK9r7s2q/GpY0h+CF+aJG3Wv6/jQSJ6arw9kvnDuaj7MQzlwv1GZPkOynAYawuhWVpT2I37i6vdiFgtyddnx5XnIYzWUgc4aJHzFTcx9P6C0FswEcPHMFo5Gr9gnJ6cVQKgIWqIy5YGH9d+N6MIG9On1ad5WFuTXXKriXmlYBcF7iWcYs1I9mOSBzuq35upfVeTvdQ96PkhPskXavGhMQjNlYDoPeRJljFiH3RyquQa7DIi34rCG42N8U+1X+qufwD079lZ24Rr0YhG3eVQnS0gCBVSzcL/r67oEYvF82oTYjhuZ8q4v2MquU6ZlyvyzDe8NGX4w98SaKcNYJZvGRwWjSoUbeA==~3293506~3424568; aus=1; _CASE_=7a23604865233b233735343430323935232d23624865233b3036392d23654865233b333338332d236d547165233b23333133342c31302c33325530383b33313b31382a31363b3131232d236d6075233b232c372f33333536313738232d236d636d233b236c606d606a6021626e6f757378216472756075642176687360232d236d6e6f66233b233031372f383531303634232d2371426e233b233032353731232d23724865233b30303432313436322d237255787164233b236e6e62232d23764865233b312d23766972233b235a5c232d2376697272233b235a5c237c; _UUID_CAS_=111b89b3-7dd0-4623-bcea-ea29f2f2fa83; _dc_gtm_UA-126956641-6=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.87ba59134277b4588a8e6b3a82f5996b.1737634766884.1737634766884.1737634834931.4%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.82e320e1c5f354d0e9b89cbb7e241b2c.1737634774733.1737634774733.1737634805183.1%22%7D; _dc_gtm_UA-9801603-1=1; _ga_70947XW48P=GS1.1.1737634766.1.1.1737634835.59.0.0; _ga=GA1.1.1746878262.1737634767; webauthn-session=2a65e883-2aa4-4fff-88bd-d292d9199d26",
            "_UUID_NONLOGIN_=8f99d5dd839ad1780723e8595ad38987; _UUID_NONLOGIN_.sig=ciTgSOoFDossyyjCLdfGFWxNt2w; _gcl_au=1.1.331787603.1737412981; _SID_Tokopedia_=rE4HKUspgx3YEb4M6zHuh7FPxz7hEsGH2ilCP1heDP2T_SJ2uXv4d-NBiIN2v_xzAm39GiewdOM2V8GPEuWJttUV1m23WNsCqBNtPOTd-3gHZKhh1YOgff3lIHo1fIeO; DID=3c43b6b28f92625d9d7b390ba99c537c62a4d87462943c414f7eab7fe94efcc3e7886ac9c2aafbef560d9fde2b03553a; DID_JS=M2M0M2I2YjI4ZjkyNjI1ZDlkN2IzOTBiYTk5YzUzN2M2MmE0ZDg3NDYyOTQzYzQxNGY3ZWFiN2ZlOTRlZmNjM2U3ODg2YWM5YzJhYWZiZWY1NjBkOWZkZTJiMDM1NTNh47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.1786514285.1737412982; ak_bmsc=2370BD7F56AE72218B590E7172AFB6E0~000000000000000000000000000000~YAAQ5i2pyiq8GmmUAQAAqzjihRqEn1r+H8D382KNWkQF2Q5qQrV56FrBezdinISsra53rNILM2p7daQBGJhXVkQ/8NZhkZR/BsDP75R7yFtjEM0Rfjz0Tk/YpKszleH3jUNiaJT2qYkya/MH3o7st6fdrWBnZ1OtMWwipJXnpGyhJjcfbcgivGC3pbBNjtJQNZoRo0V3elQcSBAsJBvh7MtEo9vHEDxWyrL/f+a4V8+nbJih3KV+ANthkLcYWaSf7RA6ThQ57CgwcXyn5chcztUHYoJneAwI/Spa1sj6xR5ZqTRb4iaKSPblRxWY98itGrEhxasWFmsTgzhc7Sa9V8Smm9wkTkCAGKJQuPxx7l8ozJ7wLAlPIUkNequHEoVgpYrlgXYa8APW25wsEMgqCzeEqXQ4cxDTJQ+CKdl1DillrL3qTpNDj37JLcs5uMx4eOOuAmAcBVlYHMa7HKBB; AMP_TOKEN=%24NOT_FOUND; _tt_enable_cookie=1; _ttp=9ko8J_DU5G9xwpTPQmCcci2-EAb.tt.1; TOPATK=MkwobnZbQCujAywkR-WEmg; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImZhYjYyOGQ2LTFhMmQtNWM2NS1iY2E0LWI3ZmVkMTQ2YjY4MSIsImNyZWF0ZWQiOjE3Mzc0MTMwMDc5NTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=RKzymP3PuU0ExKH5fb+TNGxXZ1glgdsMuHH0FJ3NktJuXxpF; aus=1; _CASE_=2e77341c31776f776361606064666d61777977361c31776f64626d7977311c31776f67676c67797739002531776f77676567607865647867640165606f61666f60637e65626f6565777977393421776f7778637b6767616265636c777977393739776f77383439343e3475363a3b21272c7530262134213075223c2734777977393a3b32776f776465637b6c616564626077797725163a776f776466616365777977261c31776f6464606665606266797726012c2530776f773a3a36777977221c31776f657977223d26776f770e08777977223d2626776f770e087728; _UUID_CAS_=9e9d1a0a-09c5-419f-a24b-dd1adcdbf1e6; _fbp=fb.1.1737413098523.68838001380503701; bm_sv=23F216B8BDF571EE83DBA5B0205B1B79~YAAQBJ9OcGCHKWmUAQAAD3L+hRp/QT8vjTsgrbBZ5HS3EOHVKh5GKeTybqhtIBGY/XVGwcatr669ugRz0hepTftu/NbWvUO2Kj7Q9Z01mC9KIDWhN/QNRGiMxbWynQ+qYvjGkKzrlgCaIC7DL45uUd1tB/XI95GaZ4259TRvGsKmOVaA34zhkM+/oiLiPc6oY8rR1/6yTQNlh1zczaE93WjKGosbUdkE2SYt659RWL5FMwQCJ8HGGsj/VI+6dcuzZVwklA==~1; bm_sz=5AFD8A5B2276517CD536C357A5D9F68D~YAAQCOnOF6exDoWUAQAA54D+hRo7rcYdyWKCymwMXzaP7HwNEPgrDEuRSaF519576hoiPLmx0FPlK6IIWkFg5LJAS+6743VHjN+bxYYnezxjHSGsFnsRa/kSij0lP9iETUT7VRD2BFDEbiiCYaS13AHEaADsYxf6bB4i73YmaG6e0tLw/ox92OJUliJ3WsB2zpVOmyTRTCJVh7OFOHl/onLXDBJpGo9WTPjifXMCBvcSStbld0kQMXuT7rwOocRclOOoQ2Ltp7UHA3R+X5vtG+LnYzg/x8SdSnWVXDLv5uLy5v4fY1qihi2PYTxRlt3whXXxdlN8t8X5/mLqf2bMQuJj3Y1u3kmrP5miU9SpZfskrDGgU8QeVCXycK4Vv5eL0lzjhVPI++BlCfhvt45IXH7NoABweLDEqhzN2g+gsI/Hq99pZCLAvEEJMo86IQb7pqo++yq5xm1+PwaIjWFIuRQB+bsZ1IkR3BMTh2v1ke1P4oTT/54sBKLuz0c39NzNEBKkk2uJuBTneIpE1/V6j8AqcAZwhMHwCAMe8GvwRk3S6YSz4lGJgh3wGhbmUnYI~3622200~3552837; _abck=5BD2CFC94D1BBDB7FBD23F6C7830515B~0~YAAQ5i2pyqIHG2mUAQAAhh0Phg3gcdyujPqux1AKOb2fB96tgAMucrNLZQWocTsgI+tOxBnoDsDJmksfeDUyNbGpPZrOFzbJ/fUxTFNPxtqGhIjrfVHekV9WcwFsk8oV81DFOEvxJ7TZMXkplEs129HNAcwo3bEdCALjW3nT3t/8yGQRl5PHe6eXoHX4kIQXYGPZkoFbDQjIQwaYM4T0lrqX+Im+msBvP7o6WBrtGfY7rA61OshvcobQ/MU7iUaMYA6R1NHQsQ0Zs/Ev79RPFF04oq4eEnqrgKBkzaAdjJplPv57xJ2VTIbEGLwke/eEWDN+lSLVGZl+vKUnz0TkQtsEKhDDwuFflOgYKyngQSybgyzWs0ErbsZFEeSudgn/wvS3phG4qoOdlhrq18yYEh7xMZQNUWx/YzVbO6dMQ1WqWHP9Cjo/z7w7rAbvxSAe8t6VvMMBkiiAgPYWhuYfQzIC1OyNvcRKOso/5/tFuMW+3wcz76zCrfMp+yt9ih60NHuXCz6o8cJOScdKqCZNDO8Q0tq9F2jMxpV2f96YduQ6UeAmWJxiHnDvxqOx9LNrMddpdCyAwgWrwaq7raV5/PRq/OUtTLNWtIYSMg+6ckf0UEY4GUtQ8gbJR7CBRcWgbYjlm7eZkrt1KzNXDgCidqEzgNDT1ySt/eLkR/v57yElZ8sEzKJkx+uiB9SRVIBbLcwfqMvRywQtfyPvKG4=~-1~-1~-1; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85eb825b3d925425e8af813e0d7c97b5.1737412981788.1737412981788.1737415929496.9%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.8e22ef5e53b67456ebb5f5f1684e5283.1737413008027.1737413008027.1737413033102.1%22%7D; _ga_70947XW48P=GS1.1.1737412981.1.1.1737415929.55.0.0; _ga=GA1.1.1911615655.1737412982; webauthn-session=a5036154-6256-48a0-baa1-6d1d4ead4ef9",
           "_UUID_NONLOGIN_=8f99d5dd839ad1780723e8595ad38987; _UUID_NONLOGIN_.sig=ciTgSOoFDossyyjCLdfGFWxNt2w; _gcl_au=1.1.331787603.1737412981; _SID_Tokopedia_=rE4HKUspgx3YEb4M6zHuh7FPxz7hEsGH2ilCP1heDP2T_SJ2uXv4d-NBiIN2v_xzAm39GiewdOM2V8GPEuWJttUV1m23WNsCqBNtPOTd-3gHZKhh1YOgff3lIHo1fIeO; DID=3c43b6b28f92625d9d7b390ba99c537c62a4d87462943c414f7eab7fe94efcc3e7886ac9c2aafbef560d9fde2b03553a; DID_JS=M2M0M2I2YjI4ZjkyNjI1ZDlkN2IzOTBiYTk5YzUzN2M2MmE0ZDg3NDYyOTQzYzQxNGY3ZWFiN2ZlOTRlZmNjM2U3ODg2YWM5YzJhYWZiZWY1NjBkOWZkZTJiMDM1NTNh47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.1786514285.1737412982; ak_bmsc=2370BD7F56AE72218B590E7172AFB6E0~000000000000000000000000000000~YAAQ5i2pyiq8GmmUAQAAqzjihRqEn1r+H8D382KNWkQF2Q5qQrV56FrBezdinISsra53rNILM2p7daQBGJhXVkQ/8NZhkZR/BsDP75R7yFtjEM0Rfjz0Tk/YpKszleH3jUNiaJT2qYkya/MH3o7st6fdrWBnZ1OtMWwipJXnpGyhJjcfbcgivGC3pbBNjtJQNZoRo0V3elQcSBAsJBvh7MtEo9vHEDxWyrL/f+a4V8+nbJih3KV+ANthkLcYWaSf7RA6ThQ57CgwcXyn5chcztUHYoJneAwI/Spa1sj6xR5ZqTRb4iaKSPblRxWY98itGrEhxasWFmsTgzhc7Sa9V8Smm9wkTkCAGKJQuPxx7l8ozJ7wLAlPIUkNequHEoVgpYrlgXYa8APW25wsEMgqCzeEqXQ4cxDTJQ+CKdl1DillrL3qTpNDj37JLcs5uMx4eOOuAmAcBVlYHMa7HKBB; _tt_enable_cookie=1; _ttp=9ko8J_DU5G9xwpTPQmCcci2-EAb.tt.1; TOPATK=MkwobnZbQCujAywkR-WEmg; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImZhYjYyOGQ2LTFhMmQtNWM2NS1iY2E0LWI3ZmVkMTQ2YjY4MSIsImNyZWF0ZWQiOjE3Mzc0MTMwMDc5NTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=RKzymP3PuU0ExKH5fb+TNGxXZ1glgdsMuHH0FJ3NktJuXxpF; aus=1; _CASE_=2e77341c31776f776361606064666d61777977361c31776f64626d7977311c31776f67676c67797739002531776f77676567607865647867640165606f61666f60637e65626f6565777977393421776f7778637b6767616265636c777977393739776f77383439343e3475363a3b21272c7530262134213075223c2734777977393a3b32776f776465637b6c616564626077797725163a776f776466616365777977261c31776f6464606665606266797726012c2530776f773a3a36777977221c31776f657977223d26776f770e08777977223d2626776f770e087728; _UUID_CAS_=9e9d1a0a-09c5-419f-a24b-dd1adcdbf1e6; _fbp=fb.1.1737413098523.68838001380503701; bm_sv=23F216B8BDF571EE83DBA5B0205B1B79~YAAQBJ9OcGCHKWmUAQAAD3L+hRp/QT8vjTsgrbBZ5HS3EOHVKh5GKeTybqhtIBGY/XVGwcatr669ugRz0hepTftu/NbWvUO2Kj7Q9Z01mC9KIDWhN/QNRGiMxbWynQ+qYvjGkKzrlgCaIC7DL45uUd1tB/XI95GaZ4259TRvGsKmOVaA34zhkM+/oiLiPc6oY8rR1/6yTQNlh1zczaE93WjKGosbUdkE2SYt659RWL5FMwQCJ8HGGsj/VI+6dcuzZVwklA==~1; bm_sz=5AFD8A5B2276517CD536C357A5D9F68D~YAAQCOnOF6exDoWUAQAA54D+hRo7rcYdyWKCymwMXzaP7HwNEPgrDEuRSaF519576hoiPLmx0FPlK6IIWkFg5LJAS+6743VHjN+bxYYnezxjHSGsFnsRa/kSij0lP9iETUT7VRD2BFDEbiiCYaS13AHEaADsYxf6bB4i73YmaG6e0tLw/ox92OJUliJ3WsB2zpVOmyTRTCJVh7OFOHl/onLXDBJpGo9WTPjifXMCBvcSStbld0kQMXuT7rwOocRclOOoQ2Ltp7UHA3R+X5vtG+LnYzg/x8SdSnWVXDLv5uLy5v4fY1qihi2PYTxRlt3whXXxdlN8t8X5/mLqf2bMQuJj3Y1u3kmrP5miU9SpZfskrDGgU8QeVCXycK4Vv5eL0lzjhVPI++BlCfhvt45IXH7NoABweLDEqhzN2g+gsI/Hq99pZCLAvEEJMo86IQb7pqo++yq5xm1+PwaIjWFIuRQB+bsZ1IkR3BMTh2v1ke1P4oTT/54sBKLuz0c39NzNEBKkk2uJuBTneIpE1/V6j8AqcAZwhMHwCAMe8GvwRk3S6YSz4lGJgh3wGhbmUnYI~3622200~3552837; _abck=5BD2CFC94D1BBDB7FBD23F6C7830515B~0~YAAQ5i2pyqIHG2mUAQAAhh0Phg3gcdyujPqux1AKOb2fB96tgAMucrNLZQWocTsgI+tOxBnoDsDJmksfeDUyNbGpPZrOFzbJ/fUxTFNPxtqGhIjrfVHekV9WcwFsk8oV81DFOEvxJ7TZMXkplEs129HNAcwo3bEdCALjW3nT3t/8yGQRl5PHe6eXoHX4kIQXYGPZkoFbDQjIQwaYM4T0lrqX+Im+msBvP7o6WBrtGfY7rA61OshvcobQ/MU7iUaMYA6R1NHQsQ0Zs/Ev79RPFF04oq4eEnqrgKBkzaAdjJplPv57xJ2VTIbEGLwke/eEWDN+lSLVGZl+vKUnz0TkQtsEKhDDwuFflOgYKyngQSybgyzWs0ErbsZFEeSudgn/wvS3phG4qoOdlhrq18yYEh7xMZQNUWx/YzVbO6dMQ1WqWHP9Cjo/z7w7rAbvxSAe8t6VvMMBkiiAgPYWhuYfQzIC1OyNvcRKOso/5/tFuMW+3wcz76zCrfMp+yt9ih60NHuXCz6o8cJOScdKqCZNDO8Q0tq9F2jMxpV2f96YduQ6UeAmWJxiHnDvxqOx9LNrMddpdCyAwgWrwaq7raV5/PRq/OUtTLNWtIYSMg+6ckf0UEY4GUtQ8gbJR7CBRcWgbYjlm7eZkrt1KzNXDgCidqEzgNDT1ySt/eLkR/v57yElZ8sEzKJkx+uiB9SRVIBbLcwfqMvRywQtfyPvKG4=~-1~-1~-1; AMP_TOKEN=%24NOT_FOUND; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85eb825b3d925425e8af813e0d7c97b5.1737412981788.1737412981788.1737416671574.11%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.8e22ef5e53b67456ebb5f5f1684e5283.1737413008027.1737413008027.1737413033102.1%22%7D; _ga_70947XW48P=GS1.1.1737412981.1.1.1737416671.41.0.0; _ga=GA1.1.1911615655.1737412982; webauthn-session=a7290a99-02a2-4e60-9cba-44a01c5d3715",
            "_UUID_NONLOGIN_=8f99d5dd839ad1780723e8595ad38987; _UUID_NONLOGIN_.sig=ciTgSOoFDossyyjCLdfGFWxNt2w; _gcl_au=1.1.331787603.1737412981; _SID_Tokopedia_=rE4HKUspgx3YEb4M6zHuh7FPxz7hEsGH2ilCP1heDP2T_SJ2uXv4d-NBiIN2v_xzAm39GiewdOM2V8GPEuWJttUV1m23WNsCqBNtPOTd-3gHZKhh1YOgff3lIHo1fIeO; DID=3c43b6b28f92625d9d7b390ba99c537c62a4d87462943c414f7eab7fe94efcc3e7886ac9c2aafbef560d9fde2b03553a; DID_JS=M2M0M2I2YjI4ZjkyNjI1ZDlkN2IzOTBiYTk5YzUzN2M2MmE0ZDg3NDYyOTQzYzQxNGY3ZWFiN2ZlOTRlZmNjM2U3ODg2YWM5YzJhYWZiZWY1NjBkOWZkZTJiMDM1NTNh47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.1786514285.1737412982; ak_bmsc=2370BD7F56AE72218B590E7172AFB6E0~000000000000000000000000000000~YAAQ5i2pyiq8GmmUAQAAqzjihRqEn1r+H8D382KNWkQF2Q5qQrV56FrBezdinISsra53rNILM2p7daQBGJhXVkQ/8NZhkZR/BsDP75R7yFtjEM0Rfjz0Tk/YpKszleH3jUNiaJT2qYkya/MH3o7st6fdrWBnZ1OtMWwipJXnpGyhJjcfbcgivGC3pbBNjtJQNZoRo0V3elQcSBAsJBvh7MtEo9vHEDxWyrL/f+a4V8+nbJih3KV+ANthkLcYWaSf7RA6ThQ57CgwcXyn5chcztUHYoJneAwI/Spa1sj6xR5ZqTRb4iaKSPblRxWY98itGrEhxasWFmsTgzhc7Sa9V8Smm9wkTkCAGKJQuPxx7l8ozJ7wLAlPIUkNequHEoVgpYrlgXYa8APW25wsEMgqCzeEqXQ4cxDTJQ+CKdl1DillrL3qTpNDj37JLcs5uMx4eOOuAmAcBVlYHMa7HKBB; _tt_enable_cookie=1; _ttp=9ko8J_DU5G9xwpTPQmCcci2-EAb.tt.1; TOPATK=MkwobnZbQCujAywkR-WEmg; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImZhYjYyOGQ2LTFhMmQtNWM2NS1iY2E0LWI3ZmVkMTQ2YjY4MSIsImNyZWF0ZWQiOjE3Mzc0MTMwMDc5NTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=RKzymP3PuU0ExKH5fb+TNGxXZ1glgdsMuHH0FJ3NktJuXxpF; aus=1; _CASE_=2e77341c31776f776361606064666d61777977361c31776f64626d7977311c31776f67676c67797739002531776f77676567607865647867640165606f61666f60637e65626f6565777977393421776f7778637b6767616265636c777977393739776f77383439343e3475363a3b21272c7530262134213075223c2734777977393a3b32776f776465637b6c616564626077797725163a776f776466616365777977261c31776f6464606665606266797726012c2530776f773a3a36777977221c31776f657977223d26776f770e08777977223d2626776f770e087728; _UUID_CAS_=9e9d1a0a-09c5-419f-a24b-dd1adcdbf1e6; _fbp=fb.1.1737413098523.68838001380503701; AMP_TOKEN=%24NOT_FOUND; _gat_UA-9801603-1=1; _abck=5BD2CFC94D1BBDB7FBD23F6C7830515B~0~YAAQEunOF5QG9XSUAQAAQYsnhg0mj05Y9Ang4qM7rjJrlPk3ON2Z0vVmNhUW4miM6RTVvT+NX6UCRPmFVm15V3r/RDnPIJTKSK8iu9Onw0VEtun0whzFLhIduU/ZISxnEiO8BfCNOwJex37wv/p5S3LALpsFdX93MNfpXJy1cpXtYNV/SIY7G2hLGQf66KXojR/+3uTSVA0XUrSqOYQrC2w49T1vydH2YhNJP1dyuyV2W0DrW6OirhXT8lQndyMnDtQzj2CZTMnTnTlZ1LOki44NEwfxbwAZ69awNWFX4cSVznCgMitEW1uSd2ND5eCNcmZcuHA77kVLK9aKaFZTuL1K9hbqObR6q3GV6EQhZ6uwmT5eU17Ge28mK0t591RpIeZ4I35O6WFa8eH/zr4Px474UCcigvOFlNEzRDBJXhAIdfpMsTPLyppy6OJq0ejbcnK+fBif5qe5ClsYOaYpAfvl0D/yCXYT1Rj8PiDNRi/0wYLNLkf5Ud10VUOc3tYB54mNda6Dp6wMemsata20eD3Ies94wqaTIjzl6c2342zPZzkCd1VzGwWTM/8BOqk4Nd6v7o1zzK9jd5upXelhrYAqa5HNrdfxAjAHj34DNwosSvs14Lg02cPvtBAm2ZxbtbdcWvmmgbQF7B1EjKZEaHbx4sdgCW6Ohk8=~-1~-1~-1; bm_sz=5AFD8A5B2276517CD536C357A5D9F68D~YAAQEunOF5UG9XSUAQAAQYsnhho4YAAj/sWPjXPOtJsEFS57OSLbkSqZJfLozRp3bZE2AZDs4NoE2K0ogtWNosqXqKFB7gZSsfBRhpRGyuq+xObAkjfZuREjA7YazmLZlyB3JOpUdSrMhBII1mx5E0kOI1DpwaoXuLh0GFXnMq+vo5HgBRJdnQnbXJMjj5C3GAYmOJb/o/RyZ2K9CU/8zbGe7r5mbenr0gF0SvGyNjYTzXbLeci6CU8eDWigM8yFPCv5R2OVoLRqwNUGrjiyjvLEs2c/S+17CZglH/2XK0vNlzL5QL4NgQe/Grh0zG43dOmKUQditnkSte4BhcIzE40JaGQDMkr2OguJ2yN4HTzK8QQN1WH/JeR0bLFVW/I1Jyc9cNCyACtnUf3oc3gawrmjM3s/LIW06ohyOAJRZqdQtqVdXiNVjq5OwgMEmHA5Loau+XW58N+lNjN+Pm0ZmWXsBrkNv3F1IKLcUYZiAutbh6xiaKf73boA9tHzw66/hzRpqhY1nHsktyHQdW6Eh4TYQNsKIVL9UKoGWxAHsnlL34H7/IrY3yK4hwYEfKVNexrgHA==~3622200~3552837; bm_sv=23F216B8BDF571EE83DBA5B0205B1B79~YAAQ1i2pyq+0+WiUAQAA8SQohhq+dBN7E22saUZIqzDKq5JEYOnOTTwgG7co9xZo2clMvQxsVR5S1HewinhBJdl4HqX7VgHyrqrLc6P7qqp54A+oYlYgwlkhbTNZg5zONB3iHPM5ViSgcmz6Ok6/htU+4I5Dyz5rQqN1RLbKusniDdvpeWSIzHBUPRejwbUuhjlFHR1eMXOhZ5zpngkDJpDzk987UKo2AsPi/Ve1WUecFYJ68Vl5KIZf8NxKULpMWMaPSw==~1; _dc_gtm_UA-9801603-1=1; _dc_gtm_UA-126956641-6=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85eb825b3d925425e8af813e0d7c97b5.1737412981788.1737412981788.1737417566956.15%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.8e22ef5e53b67456ebb5f5f1684e5283.1737413008027.1737413008027.1737413033102.1%22%7D; _ga_70947XW48P=GS1.1.1737412981.1.1.1737417566.60.0.0; _ga=GA1.1.1911615655.1737412982; webauthn-session=9b5829fe-2023-41d2-92be-57c4ad494f38",
            "_UUID_NONLOGIN_=8f99d5dd839ad1780723e8595ad38987; _UUID_NONLOGIN_.sig=ciTgSOoFDossyyjCLdfGFWxNt2w; _gcl_au=1.1.331787603.1737412981; _SID_Tokopedia_=rE4HKUspgx3YEb4M6zHuh7FPxz7hEsGH2ilCP1heDP2T_SJ2uXv4d-NBiIN2v_xzAm39GiewdOM2V8GPEuWJttUV1m23WNsCqBNtPOTd-3gHZKhh1YOgff3lIHo1fIeO; DID=3c43b6b28f92625d9d7b390ba99c537c62a4d87462943c414f7eab7fe94efcc3e7886ac9c2aafbef560d9fde2b03553a; DID_JS=M2M0M2I2YjI4ZjkyNjI1ZDlkN2IzOTBiYTk5YzUzN2M2MmE0ZDg3NDYyOTQzYzQxNGY3ZWFiN2ZlOTRlZmNjM2U3ODg2YWM5YzJhYWZiZWY1NjBkOWZkZTJiMDM1NTNh47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.1786514285.1737412982; ak_bmsc=2370BD7F56AE72218B590E7172AFB6E0~000000000000000000000000000000~YAAQ5i2pyiq8GmmUAQAAqzjihRqEn1r+H8D382KNWkQF2Q5qQrV56FrBezdinISsra53rNILM2p7daQBGJhXVkQ/8NZhkZR/BsDP75R7yFtjEM0Rfjz0Tk/YpKszleH3jUNiaJT2qYkya/MH3o7st6fdrWBnZ1OtMWwipJXnpGyhJjcfbcgivGC3pbBNjtJQNZoRo0V3elQcSBAsJBvh7MtEo9vHEDxWyrL/f+a4V8+nbJih3KV+ANthkLcYWaSf7RA6ThQ57CgwcXyn5chcztUHYoJneAwI/Spa1sj6xR5ZqTRb4iaKSPblRxWY98itGrEhxasWFmsTgzhc7Sa9V8Smm9wkTkCAGKJQuPxx7l8ozJ7wLAlPIUkNequHEoVgpYrlgXYa8APW25wsEMgqCzeEqXQ4cxDTJQ+CKdl1DillrL3qTpNDj37JLcs5uMx4eOOuAmAcBVlYHMa7HKBB; _tt_enable_cookie=1; _ttp=9ko8J_DU5G9xwpTPQmCcci2-EAb.tt.1; TOPATK=MkwobnZbQCujAywkR-WEmg; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImZhYjYyOGQ2LTFhMmQtNWM2NS1iY2E0LWI3ZmVkMTQ2YjY4MSIsImNyZWF0ZWQiOjE3Mzc0MTMwMDc5NTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=RKzymP3PuU0ExKH5fb+TNGxXZ1glgdsMuHH0FJ3NktJuXxpF; aus=1; _CASE_=2e77341c31776f776361606064666d61777977361c31776f64626d7977311c31776f67676c67797739002531776f77676567607865647867640165606f61666f60637e65626f6565777977393421776f7778637b6767616265636c777977393739776f77383439343e3475363a3b21272c7530262134213075223c2734777977393a3b32776f776465637b6c616564626077797725163a776f776466616365777977261c31776f6464606665606266797726012c2530776f773a3a36777977221c31776f657977223d26776f770e08777977223d2626776f770e087728; _UUID_CAS_=9e9d1a0a-09c5-419f-a24b-dd1adcdbf1e6; _fbp=fb.1.1737413098523.68838001380503701; AMP_TOKEN=%24NOT_FOUND; _abck=5BD2CFC94D1BBDB7FBD23F6C7830515B~0~YAAQE+nOFwmjoHWUAQAA0xEshg3H/DdlfkA3HYlkRbz5hdwp2vDzw0WTBBniSd9lP/qDmFSOd5AmA+33hyUaAyGhy9syP67J1J+qF6Xp+7Jzb7b6UELM839g3k6QQQl9l2UVx1/ibR7OGaloWv0/jFqnKeWTdpkZNCRsTcCS7+sPXZexf21Nfmd1f/vwm7G1qNvSAfBc2zlFeHRfDc7vW+NK3jwG/fPNoUeoLloJJT2FlPodcHtw7SH32A4Ijfg0UFJeUTEIhY9iLUCThYf78vZOR630FGwAmdXY6tHfRHsudAQWuHXgi9Ek1b/KjKd27CTdPqNWwB+JcDYoaZLKKFzZlcGnjNdIRhRKuAli1Mu1EDPUQLuFEAgErSQcJM7id3Uwit6pFWjhJF/mAAXmffzxVfXzC6cIbi720gHaiCT+ulgpyf7XAhnZSLLTMVtZJC/SC06DqxzIPtMLWt0D5cbPu4VjQ1IzxcTrWFdBZLVRfLeAWBTuMeT3VuMWXtFBcuV7uTMXUoyqMLGa1OaoNQdsc6cX/gyFW+qEIQYyurY6ZNObMBHDRzz739JfJ1ng/ov5QTxgbyNuxCUpF7O6S1YL22a1HX+kktRHvnYgELAWzw6VDf/ACV3t3gul3Tvlvh1R66Dpp+JiWaaUJqyN1lkwrEUrUZCfPDo=~-1~-1~-1; bm_sz=5AFD8A5B2276517CD536C357A5D9F68D~YAAQE+nOFwqjoHWUAQAA0xEshhocqAjR5TwM/b6DYUlkifT4002HQFcNQFjs+LV4zUtgLs62zYpUbo1s46fv/zc7mul7xi9r7EtL8P7YNMXcAJUvl87y/6wfW/WtsbysxzQFlu5idYnf6p7Fir8rusCcEfKnt+sSGEFsnsjem+ydZJGA2HA4ba1P0Y1dzXwcidZJJw9w6epFgvNXQshN/KPsF4pb1Z51AjB6ym38aMER74yVWjhSg/kS2YVHGvyQIiPxB7TbmRaSZGl3gkHY9Tr0/okvdN7U7SDgt/S9kxM2/lIpl+nml2k8zF/U9tFYjm9VK5a+5TNBAl6C+Jz5DXILJyG/QWAH7SoMfIsOjyYGFtUyQ9ruGm3EfwunMXDT5tWk0SY/ekveIjhQQtRxns62IaxxBUR0lvqk3uOuuvfkVKQyPgLheWPKvnPJTca+Tkik/KE5pCTFFIsGy296SZUMKe15M0Lgr1JadBmI1kAerOLrUzLwTjyxGo5JB0jAQQZ2aRpIbViye7hvEoGpARdJ83KnfzPav/dK11PIW3Mzh79bLD1rAG6ERXqTdg2emiicjrSrlg==~3622200~3552837; bm_sv=23F216B8BDF571EE83DBA5B0205B1B79~YAAQ1i2pyqXJ+WiUAQAAGE0uhhpH+Z8MdDCA+HoQmXLw8EjPpCzFi/kzBSe1Sx8fyk2NZtcjdHwb05U9UlhMcUUgABGWC5BuWRYsDk74g46VSqEAgjrBifU6/Pfx18isdxS44HAfsxBVnT36CQrPoSEX54QpvImZ+4OBwDqNmsby6+E8LP+wNnrw6AKG8wEnaD4+f8ig+r2NPyKO7VpN/izStYkyotRO0lXFt/C1cmwA+FKHCIW5Ga7y1lzwkydAKMxT5A==~1; _dc_gtm_UA-126956641-6=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85eb825b3d925425e8af813e0d7c97b5.1737412981788.1737412981788.1737419141703.17%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.8e22ef5e53b67456ebb5f5f1684e5283.1737413008027.1737413008027.1737413033102.1%22%7D; _dc_gtm_UA-9801603-1=1; _ga_70947XW48P=GS1.1.1737412981.1.1.1737419141.60.0.0; _ga=GA1.1.1911615655.1737412982; webauthn-session=35155137-d279-46fd-9f93-2cd0e963e08e",
            "_UUID_NONLOGIN_=8f99d5dd839ad1780723e8595ad38987; _UUID_NONLOGIN_.sig=ciTgSOoFDossyyjCLdfGFWxNt2w; _gcl_au=1.1.331787603.1737412981; _SID_Tokopedia_=rE4HKUspgx3YEb4M6zHuh7FPxz7hEsGH2ilCP1heDP2T_SJ2uXv4d-NBiIN2v_xzAm39GiewdOM2V8GPEuWJttUV1m23WNsCqBNtPOTd-3gHZKhh1YOgff3lIHo1fIeO; DID=3c43b6b28f92625d9d7b390ba99c537c62a4d87462943c414f7eab7fe94efcc3e7886ac9c2aafbef560d9fde2b03553a; DID_JS=M2M0M2I2YjI4ZjkyNjI1ZDlkN2IzOTBiYTk5YzUzN2M2MmE0ZDg3NDYyOTQzYzQxNGY3ZWFiN2ZlOTRlZmNjM2U3ODg2YWM5YzJhYWZiZWY1NjBkOWZkZTJiMDM1NTNh47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.1786514285.1737412982; ak_bmsc=2370BD7F56AE72218B590E7172AFB6E0~000000000000000000000000000000~YAAQ5i2pyiq8GmmUAQAAqzjihRqEn1r+H8D382KNWkQF2Q5qQrV56FrBezdinISsra53rNILM2p7daQBGJhXVkQ/8NZhkZR/BsDP75R7yFtjEM0Rfjz0Tk/YpKszleH3jUNiaJT2qYkya/MH3o7st6fdrWBnZ1OtMWwipJXnpGyhJjcfbcgivGC3pbBNjtJQNZoRo0V3elQcSBAsJBvh7MtEo9vHEDxWyrL/f+a4V8+nbJih3KV+ANthkLcYWaSf7RA6ThQ57CgwcXyn5chcztUHYoJneAwI/Spa1sj6xR5ZqTRb4iaKSPblRxWY98itGrEhxasWFmsTgzhc7Sa9V8Smm9wkTkCAGKJQuPxx7l8ozJ7wLAlPIUkNequHEoVgpYrlgXYa8APW25wsEMgqCzeEqXQ4cxDTJQ+CKdl1DillrL3qTpNDj37JLcs5uMx4eOOuAmAcBVlYHMa7HKBB; _tt_enable_cookie=1; _ttp=9ko8J_DU5G9xwpTPQmCcci2-EAb.tt.1; TOPATK=MkwobnZbQCujAywkR-WEmg; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImZhYjYyOGQ2LTFhMmQtNWM2NS1iY2E0LWI3ZmVkMTQ2YjY4MSIsImNyZWF0ZWQiOjE3Mzc0MTMwMDc5NTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=RKzymP3PuU0ExKH5fb+TNGxXZ1glgdsMuHH0FJ3NktJuXxpF; aus=1; _CASE_=2e77341c31776f776361606064666d61777977361c31776f64626d7977311c31776f67676c67797739002531776f77676567607865647867640165606f61666f60637e65626f6565777977393421776f7778637b6767616265636c777977393739776f77383439343e3475363a3b21272c7530262134213075223c2734777977393a3b32776f776465637b6c616564626077797725163a776f776466616365777977261c31776f6464606665606266797726012c2530776f773a3a36777977221c31776f657977223d26776f770e08777977223d2626776f770e087728; _UUID_CAS_=9e9d1a0a-09c5-419f-a24b-dd1adcdbf1e6; _fbp=fb.1.1737413098523.68838001380503701; AMP_TOKEN=%24NOT_FOUND; _abck=5BD2CFC94D1BBDB7FBD23F6C7830515B~0~YAAQE+nOFwmjoHWUAQAA0xEshg3H/DdlfkA3HYlkRbz5hdwp2vDzw0WTBBniSd9lP/qDmFSOd5AmA+33hyUaAyGhy9syP67J1J+qF6Xp+7Jzb7b6UELM839g3k6QQQl9l2UVx1/ibR7OGaloWv0/jFqnKeWTdpkZNCRsTcCS7+sPXZexf21Nfmd1f/vwm7G1qNvSAfBc2zlFeHRfDc7vW+NK3jwG/fPNoUeoLloJJT2FlPodcHtw7SH32A4Ijfg0UFJeUTEIhY9iLUCThYf78vZOR630FGwAmdXY6tHfRHsudAQWuHXgi9Ek1b/KjKd27CTdPqNWwB+JcDYoaZLKKFzZlcGnjNdIRhRKuAli1Mu1EDPUQLuFEAgErSQcJM7id3Uwit6pFWjhJF/mAAXmffzxVfXzC6cIbi720gHaiCT+ulgpyf7XAhnZSLLTMVtZJC/SC06DqxzIPtMLWt0D5cbPu4VjQ1IzxcTrWFdBZLVRfLeAWBTuMeT3VuMWXtFBcuV7uTMXUoyqMLGa1OaoNQdsc6cX/gyFW+qEIQYyurY6ZNObMBHDRzz739JfJ1ng/ov5QTxgbyNuxCUpF7O6S1YL22a1HX+kktRHvnYgELAWzw6VDf/ACV3t3gul3Tvlvh1R66Dpp+JiWaaUJqyN1lkwrEUrUZCfPDo=~-1~-1~-1; bm_sz=5AFD8A5B2276517CD536C357A5D9F68D~YAAQE+nOFwqjoHWUAQAA0xEshhocqAjR5TwM/b6DYUlkifT4002HQFcNQFjs+LV4zUtgLs62zYpUbo1s46fv/zc7mul7xi9r7EtL8P7YNMXcAJUvl87y/6wfW/WtsbysxzQFlu5idYnf6p7Fir8rusCcEfKnt+sSGEFsnsjem+ydZJGA2HA4ba1P0Y1dzXwcidZJJw9w6epFgvNXQshN/KPsF4pb1Z51AjB6ym38aMER74yVWjhSg/kS2YVHGvyQIiPxB7TbmRaSZGl3gkHY9Tr0/okvdN7U7SDgt/S9kxM2/lIpl+nml2k8zF/U9tFYjm9VK5a+5TNBAl6C+Jz5DXILJyG/QWAH7SoMfIsOjyYGFtUyQ9ruGm3EfwunMXDT5tWk0SY/ekveIjhQQtRxns62IaxxBUR0lvqk3uOuuvfkVKQyPgLheWPKvnPJTca+Tkik/KE5pCTFFIsGy296SZUMKe15M0Lgr1JadBmI1kAerOLrUzLwTjyxGo5JB0jAQQZ2aRpIbViye7hvEoGpARdJ83KnfzPav/dK11PIW3Mzh79bLD1rAG6ERXqTdg2emiicjrSrlg==~3622200~3552837; bm_sv=23F216B8BDF571EE83DBA5B0205B1B79~YAAQ1i2pyqXJ+WiUAQAAGE0uhhpH+Z8MdDCA+HoQmXLw8EjPpCzFi/kzBSe1Sx8fyk2NZtcjdHwb05U9UlhMcUUgABGWC5BuWRYsDk74g46VSqEAgjrBifU6/Pfx18isdxS44HAfsxBVnT36CQrPoSEX54QpvImZ+4OBwDqNmsby6+E8LP+wNnrw6AKG8wEnaD4+f8ig+r2NPyKO7VpN/izStYkyotRO0lXFt/C1cmwA+FKHCIW5Ga7y1lzwkydAKMxT5A==~1; _dc_gtm_UA-126956641-6=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85eb825b3d925425e8af813e0d7c97b5.1737412981788.1737412981788.1737419549421.17%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.8e22ef5e53b67456ebb5f5f1684e5283.1737413008027.1737413008027.1737413033102.1%22%7D; _dc_gtm_UA-9801603-1=1; _ga_70947XW48P=GS1.1.1737412981.1.1.1737419549.60.0.0; _ga=GA1.1.1911615655.1737412982; webauthn-session=7a923bdd-32cb-45cb-bf37-b8f374c4ccac"]
#Ada enam 'cookie' dengan urutan [dari atas ke bawah : a,b,c,d,e,f]
#ke identitas produk

In [13]:
#persiapan 'identitas produk'
idlist =  ["7576252307", "1624334615", "1624334615",  "1624323568",  "1624334615",  "2390937942" ] #enam sampel ini dari Erigo
#urutan dari kiri ke kanan : [a,b,c,d,e,f]
#ke fragmentasi

In [14]:
#fragmentasi data untuk kemudahan
[urla, urlb, urlc, urld, urle, urlf] = urllist #daftar URL referensi
[cookiea, cookieb, cookiec, cookied, cookiee, cookief] = cooklist #daftar cookie
[ida, idb, idc, idd, ide, idf] = idlist #daftar ID produk
#ke persiapan target dan kepala

# Proses Iterasi untuk data

## Persiapan

### Pembentukan kepala dan **'bentuk kueri'**

In [82]:
#Persiapan kepala dan kueri yang distandardisasi
def stdheader(url, cookie, idpr): #idpr adalah identitas kepala
  halaman_awal = 1 #di kueri
  kplstd = {'accept': '*/*',
   'accept-language': 'en-US,en;q=0.9',
   'bd-device-id': '7463084441910052353',
   'content-type': 'application/json',
   'cookie': cookie,
   'dnt': '1' ,
   'origin': asal,
   'priority': 'u=1, i',
   'referer': url,
   'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"',
   'sec-ch-ua-mobile': '?0',
   'sec-ch-ua-platform': '"Windows"',
   'sec-fetch-dest': 'empty',
   'sec-fetch-mode': 'cors',
   'sec-fetch-site': 'same-site',
   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
   'x-price-center': 'true',
   'x-source': 'tokopedia-lite',
   'x-tkpd-lite-service': 'zeus',
   'x-version': '1c2c0dd'}
  return kplstd

In [83]:
#variasi kepala untuk iterasi [dari indeks 0 ke lima dengan arah kiri ke kanan]
kplstdlist = [stdheader(urla, cookiea, ida), stdheader(urlb, cookieb, idb), stdheader(urlc, cookiec, idc), stdheader(urld, cookied, idd), stdheader(urle, cookiee, ide), stdheader(urlf, cookief, idf)]
#ke persiapan bentuk

In [84]:
#inspeksi hasil
print(kplstdlist[0])

{'accept': '*/*', 'accept-language': 'en-US,en;q=0.9', 'bd-device-id': '7463084441910052353', 'content-type': 'application/json', 'cookie': '_UUID_NONLOGIN_=e66ed06832acf756d426a493bf7bac26; _UUID_NONLOGIN_.sig=JpwY5Akhm0H8xr6doCJFqYqnNjM; _SID_Tokopedia_=NwrKP0GhH27USJ0-Kd_bzQmNQmkL6fJZ_hUy7px9yYDRZxXSMZ9QUiGnjxgVKNqDa3dKAKRs4PNb9KXl1lOu7EO1g8VHe6DT5qnKcwrmTq6LiJqkaQIVgDk7fGkvVozC; DID=5fc0d7cafdcac8e6f2b8b9cc5b7d62a618027955df14c273f58635e60983697a254bd4679379abc68b6565430baabd72; DID_JS=NWZjMGQ3Y2FmZGNhYzhlNmYyYjhiOWNjNWI3ZDYyYTYxODAyNzk1NWRmMTRjMjczZjU4NjM1ZTYwOTgzNjk3YTI1NGJkNDY3OTM3OWFiYzY4YjY1NjU0MzBiYWFiZDcy47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; ak_bmsc=94D1DCC12F6AF478887548E9837200FB~000000000000000000000000000000~YAAQ3i2pyqNTOWCUAQAAklTfgxrwDEn0XJX2byHL/KDtwxOnz0wDQ3WSTUw7KPiuWfUUSFYOULXksJd1NJ5McYwwmS68dg26nLjKNnh9F2yS8R9hiVZ7MJYUQYu6aelqotQUNSBBRv3OrLrWXevhmyD5xTz7K/eUXo6sqxWM7ZtobnCT9IYJceNWMr2/suwIJVeQeKFswI0Wy16DlcWcO/rSokBHAedyxW81Rj/vrhGk1h2EkVxKbGjNQHOizk+TF3T

### Iterasi utama

In [104]:
#Proses iterasi produk
def datagenerator (url, cookie, idpr):
  #pemangilan fungsi pertama (stdheader)
  header = stdheader(url, cookie, idpr)

  #persiapan bentuk untuk iterasi
  daftar_teks = []
  daftar_nilai = []
  daftar_label = []

  #Batas halaman untuk ulasan
  awal = 1 #dari halaman 1
  akhir = 50 #batas ulasan paling tinggi
  fquery = f'[{{"operationName":"productReviewList","variables":{{"productID":"{idpr}","page":{awal},"limit":10,"sortBy":"informative_score desc","filterBy":""}},"query":"query productReviewList($productID: String\u0021, $page: Int\u0021, $limit: Int\u0021, $sortBy: String, $filterBy: String) {{\\n  productrevGetProductReviewList(productID: $productID, page: $page, limit: $limit, sortBy: $sortBy, filterBy: $filterBy) {{\\n    productID\\n    list {{\\n      id: feedbackID\\n      variantName\\n      message\\n      productRating\\n      reviewCreateTime\\n      reviewCreateTimestamp\\n      isReportable\\n      isAnonymous\\n      imageAttachments {{\\n        attachmentID\\n        imageThumbnailUrl\\n        imageUrl\\n        __typename\\n      }}\\n      videoAttachments {{\\n        attachmentID\\n        videoUrl\\n        __typename\\n      }}\\n      reviewResponse {{\\n        message\\n        createTime\\n        __typename\\n      }}\\n      user {{\\n        userID\\n        fullName\\n        image\\n        url\\n        __typename\\n      }}\\n      likeDislike {{\\n        totalLike\\n        likeStatus\\n        __typename\\n      }}\\n      stats {{\\n        key\\n        formatted\\n        count\\n        __typename\\n      }}\\n      badRatingReasonFmt\\n      __typename\\n    }}\\n    shop {{\\n      shopID\\n      name\\n      url\\n      image\\n      __typename\\n    }}\\n    hasNext\\n    totalReviews\\n    __typename\\n  }}\\n}}\\n"}}]'
  while awal <= akhir:
    query = fquery #dari masing-masing hasil
    response = requests.post(turldoma, headers=header, data=query)
    daftar_ulasan = response.json()[0]['data']['productrevGetProductReviewList']['list']
    for review in daftar_ulasan:
        daftar_teks.append(review['message'])
        daftar_nilai.append(review['productRating'])
        if review['productRating'] == 0:
            daftar_label.append('total disappointment') #hal yang mengecewakan sekali
        elif review['productRating'] == 1:
            daftar_label.append('so inferior') #hal sangat inferior
        elif review['productRating'] == 2:
            daftar_label.append('very low') #sangat rendah
        elif review['productRating'] == 3:
            daftar_label.append('average') #rata-rata
        elif review['productRating'] == 4:
            daftar_label.append('good') #sangat bagus
        elif review['productRating'] == 5:
            daftar_label.append('excelent') #sempurna
        else:
            daftar_label.append('No score') #tak mau diulas
        awal += 1
  return daftar_teks, daftar_nilai, daftar_label
#ke eksekusi untuk penggabungan

In [90]:
#Proses pembentukan daftar dari enam produk untuk disatukan (Bagian 1)
teks1, nilai1, label1 = datagenerator(urla, cookiea, ida) #ProdukA
teks2, nilai2, label2 = datagenerator(urlb, cookieb, idb) #ProdukB
teks3, nilai3, label3 = datagenerator(urlc, cookiec, idc) #ProdukC
#ke bagian 2

In [91]:
#Bagian 2
teks4, nilai4, label4 = datagenerator(urld, cookied, idd) #ProdukD
teks5, nilai5, label5 = datagenerator(urle, cookiee, ide) #ProdukE
teks6, nilai6, label6 = datagenerator(urlf, cookief, idf) #ProdukF
#ke penyatuan

### Penyatuan ke satu data

In [92]:
#inspeksi hasil [sampel 1 adalah situs A]
print(teks1) #kata-kata
print(nilai1) #nilai
print(label1) #label

['barang bagus kualitas nya top harga nya naik overall 100 dijamin ori', 'modelnya keren, bahannya bagus cuma agak kepanjangan. perlu dipermak dikit.', 'tinggi 175 bb 60 ukuran L cocok si, pas dan enak banget bahannya adem, murah lagi', 'enak di pake adem dan kece pas di badan gw. tapi kayaknya mudah lecek. Nyaman dipakai Modelnya keren 😎 Jahitannya rapi Ukuran sesuai Warnanya mantap Bahannya adem Kancing tidak mudah lepas Produk sesuai dengan foto Kualitas produk bagus', 'APAAN INI WOY KEREN BANGET KEMEJA NYA ga usa ragu kualitas Erigo ganteng banget sumpah, kemeja nya ga kaku tapi juga ga kerut bahan adem lembut halus gmn yah Kyk ga mau Di lepas kalau pakai ini kemeja. Sukses terus erigo pertahankan ya kualitas barang udh ganteng banget ini mah harga setara 200 rb. Ydh itu aja review nya', 'oke paket di terima dengan baik ,, untuk kualitas barang oke kok kak 😁 bagus ,, tapi sayang nya untuk pengiriman agak lama kak oke itu saja dari saya', 'pas, bahannya enak banget dibadan dan juga 

In [93]:
#deklarasi total ulasan
sampteks = teks1 + teks2 + teks3 + teks4 + teks5 + teks6 #bagian teks
sampnilai = nilai1 + nilai2 + nilai3 + nilai4 + nilai5 + nilai6 #bagian nilai
samplabel = label1 + label2 + label3 + label4 + label5 + label6 #bagian label
#ke penyatuan

enam ulasan ini berasal dari enam produk berasal dari Erigo

In [94]:
#penyatuan ke dataframe
dfori = pd.DataFrame({'Pesan': sampteks,'Nilai': sampnilai,'Label': samplabel})
dfkopi = dfori.copy()
#cek panjang
lendfkop = len(dfkopi)
print(lendfkop)

300


#### Inspeksi hasil data dari kesalahan

In [95]:
#cek sampel data
kepdfkopi = dfkopi.head(7)
kepdfkopi

,Pesan,Nilai,Label
0,barang bagus kualitas nya top harga nya naik o...,5,excelent
1,"modelnya keren, bahannya bagus cuma agak kepan...",5,excelent
2,"tinggi 175 bb 60 ukuran L cocok si, pas dan en...",5,excelent
3,enak di pake adem dan kece pas di badan gw. ta...,5,excelent
4,APAAN INI WOY KEREN BANGET KEMEJA NYA ga usa r...,5,excelent
5,"oke paket di terima dengan baik ,, untuk kuali...",5,excelent
6,"pas, bahannya enak banget dibadan dan juga ker...",5,excelent


In [96]:
#cek kolom
dfkopinfo = dfkopi.info()
print(dfkopinfo)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Pesan   300 non-null    object
 1   Nilai   300 non-null    int64 
 2   Label   300 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.2+ KB
None


#### Pembersihan data (bebas dari data duplikat dan kosong)

In [97]:
#Data Kosong
totkos = dfkopi.isnull().sum() #Kosong

In [98]:
#inspeksi data hilang
tothil = dfkopi.isna().sum() #Hilang
print(tothil)

Pesan    0
Nilai    0
Label    0
dtype: int64


In [99]:
#inspeksi data duplikat
dfkopi.duplicated().sum()

260

In [100]:
#Pembersihan data duplikat
dfkopi.drop_duplicates(inplace=True)
dfkopitung = dfkopi.copy()
kep2 = dfkopitung.head(7) #pasca pembersihan
print(kep2)

                                               Pesan  Nilai     Label
0  barang bagus kualitas nya top harga nya naik o...      5  excelent
1  modelnya keren, bahannya bagus cuma agak kepan...      5  excelent
2  tinggi 175 bb 60 ukuran L cocok si, pas dan en...      5  excelent
3  enak di pake adem dan kece pas di badan gw. ta...      5  excelent
4  APAAN INI WOY KEREN BANGET KEMEJA NYA ga usa r...      5  excelent
5  oke paket di terima dengan baik ,, untuk kuali...      5  excelent
6  pas, bahannya enak banget dibadan dan juga ker...      5  excelent


In [101]:
#cek panjang
lendfkop2 = len(dfkopitung)
print(lendfkop2)

40


# Persiapan tahapan ulasan

## Penataan ulang kata-kata

In [ ]:
#pembuatan daftar kata-kata bahasa gaul
kata_gaul = ['yang', 'dan', 'di', 'ke', 'dari', 'ini', 'itu', 'untuk', 'dengan', 'saya', 'ada', 'akan', 'pada',
             'seperti', 'karena', 'atau', 'jika', 'tidak', 'dalam','iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"] #daftar kata gaul
#Bahasa gaul yang populer